Лаб2 

In [1]:
import datetime
import os
import urllib.request
import pandas as pd
import glob

print("Ініціалізація завершена")

Ініціалізація завершена


In [2]:
def fetch_vhi_data(region_code, start_year=1981, end_year=2024):
    directory = 'dataset'
    if not os.path.exists(directory):
        os.makedirs(directory)

    link = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_code}&year1={start_year}&year2={end_year}&type=Mean"
    response = urllib.request.urlopen(link)

    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    filename = f'vhi_region_{region_code}_{timestamp}.csv'
    file_location = os.path.join(directory, filename)

    with open(file_location, 'wb') as f:
        f.write(response.read())

    print(f"Файл '{file_location}' успішно збережено")

In [3]:
for region in range(1, 26):
    fetch_vhi_data(region)

Файл 'dataset\vhi_region_1_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_2_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_3_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_4_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_5_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_6_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_7_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_8_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_9_2025-05-30_22-04.csv' успішно збережено
Файл 'dataset\vhi_region_10_2025-05-30_22-05.csv' успішно збережено
Файл 'dataset\vhi_region_11_2025-05-30_22-05.csv' успішно збережено
Файл 'dataset\vhi_region_12_2025-05-30_22-05.csv' успішно збережено
Файл 'dataset\vhi_region_13_2025-05-30_22-05.csv' успішно збережено
Файл 'dataset\vhi_region_14_2025-05-30_22-05.csv' успішно збережено
Файл 'dataset\vhi_region_15_2025-05-30_22-05.csv' успішно

In [4]:
regions = {
    1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
    6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська', 10: 'Кіровоградська',
    11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська',
    16: 'Рівненська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
    21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська', 25: 'Республіка Крим'
}

In [5]:
def compile_dataframe(path):
    """Об'єднання всіх CSV у єдиний DataFrame"""
    files = glob.glob(path + "/*.csv")
    columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'extra']
    data_frames = []

    for file in files:
        parts = file.split('_')
        if len(parts) < 3:
            continue
        reg_id = int(parts[2])

        df = pd.read_csv(file, header=1, names=columns)
        df.at[0, 'Year'] = df.at[0, 'Year'][9:]
        df = df.drop(df.index[-1])
        df = df.drop(df[df['VHI'] == -1].index)
        df = df.drop('extra', axis=1)
        df.insert(0, 'region_code', reg_id, True)
        df['Week'] = df['Week'].astype(int)
        data_frames.append(df)

    final_df = pd.concat(data_frames).drop_duplicates().reset_index(drop=True)
    return final_df

In [6]:
vhi_dataframe = compile_dataframe('dataset')

print(vhi_dataframe)

       region_code  Year  Week    SMN     SMT    VCI    TCI    VHI
0               10  1982     1  0.059  258.24  51.11  48.78  49.95
1               10  1982     2  0.063  261.53  55.89  38.20  47.04
2               10  1982     3  0.063  263.45  57.30  32.69  44.99
3               10  1982     4  0.061  265.10  53.96  28.62  41.29
4               10  1982     5  0.058  266.42  46.87  28.57  37.72
...            ...   ...   ...    ...     ...    ...    ...    ...
54645            9  2024    48  0.135  278.17  55.23  11.86  33.55
54646            9  2024    49  0.133  277.08  57.71  10.86  34.29
54647            9  2024    50  0.130  276.49  59.45   8.68  34.07
54648            9  2024    51  0.128  276.45  62.53   5.55  34.04
54649            9  2024    52  0.129  276.48  66.13   3.71  34.92

[54650 rows x 8 columns]


In [7]:
def analyze_region(region_code, data, period=("1982", "2024")):
    """Аналіз мінімального та максимального VHI для області"""
    subset = data[(data["Year"].between(period[0], period[1])) & (data['region_code'] == region_code)]
    region_name = regions[region_code]
    max_vhi = subset["VHI"].max()
    min_vhi = subset["VHI"].min()
    print(f"Область {region_name}: min {min_vhi}, max {max_vhi}")

In [8]:
for region in range(1, 26):
    analyze_region(region, vhi_dataframe)

Область Вінницька: min 10.68, max 83.7
Область Волинська: min 15.17, max 80.65
Область Дніпропетровська: min 15.16, max 74.92
Область Донецька: min 13.28, max 90.96
Область Житомирська: min 17.58, max 93.17
Область Закарпатська: min 6.26, max 96.18
Область Запорізька: min 18.98, max 73.35
Область Івано-Франківська: min 9.36, max 91.42
Область Київська: min 12.23, max 90.61
Область Кіровоградська: min 18.41, max 79.4
Область Луганська: min 10.6, max 80.88
Область Львівська: min 6.49, max 76.84
Область Миколаївська: min 16.36, max 84.52
Область Одеська: min 12.45, max 90.32
Область Полтавська: min 18.31, max 69.96
Область Рівненська: min 5.94, max 92.31
Область Сумська: min 5.52, max 89.14
Область Тернопільська: min 15.68, max 85.14
Область Харківська: min 20.7, max 77.45
Область Херсонська: min 8.14, max 76.94
Область Хмельницька: min 12.64, max 81.96
Область Черкаська: min 19.45, max 77.71
Область Чернівецька: min 15.8, max 72.59
Область Чернігівська: min 11.25, max 82.64
Область Респу

In [9]:
def detect_drought_years(data, threshold, drought_type, start=None, end=None):
    """Пошук років із сильною або середньою посухою"""
    if start is not None:
        data = data[data['Year'] >= start]
    if end is not None:
        data = data[data['Year'] <= end]

    drought_stats = {}
    grouped = data.groupby('Year')

    for year, group in grouped:
        total_regions = len(group['region_code'].unique())

        if drought_type == "extreme":
            affected = group[group['VHI'] < 15]['region_code'].nunique()
        elif drought_type == "moderate":
            affected = group[(group['VHI'] > 15) & (group['VHI'] < 35)]['region_code'].nunique()

        if affected / total_regions > threshold / 100:
            drought_stats[year] = (affected, round((affected / total_regions) * 100, 2))

    return drought_stats

extreme_droughts = detect_drought_years(vhi_dataframe, 10, "extreme", start='2000', end='2024')
print("Екстремальна посуха:", extreme_droughts)

moderate_droughts = detect_drought_years(vhi_dataframe, 10, "moderate", start='2000', end='2024')
print("Помірна посуха:", moderate_droughts)

Екстремальна посуха: {'2000': (6, 24.0), '2007': (4, 16.0)}
Помірна посуха: {'2000': (25, 100.0), '2001': (11, 44.0), '2002': (10, 40.0), '2003': (12, 48.0), '2004': (3, 12.0), '2005': (6, 24.0), '2006': (12, 48.0), '2007': (14, 56.0), '2008': (14, 56.0), '2009': (14, 56.0), '2010': (13, 52.0), '2011': (17, 68.0), '2012': (12, 48.0), '2013': (16, 64.0), '2014': (10, 40.0), '2015': (22, 88.0), '2016': (14, 56.0), '2017': (10, 40.0), '2018': (14, 56.0), '2019': (24, 96.0), '2020': (16, 64.0), '2021': (10, 40.0), '2022': (6, 24.0), '2023': (19, 76.0), '2024': (15, 60.0)}
